## Dataset

In [15]:
SAMPLES = 10
PHASES = 5

In [17]:
class LlvmIrDataset(torch.utils.data.Dataset):
    def __init__(self, samples=SAMPLES, phases=PHASES, evaluate: bool = False):
        
        self.env = gym.make("llvm-ic-v0")
        self.samples = samples
        self.phases = phases
        
        # self.examples = {"Inst2vec":[], "IrInstructionCountOz":[]}
        self.xs = []
        self.ys = []
        self.ts = []
        
        offset = len(env.inst2vec.vocab)
        
        for _ in range(samples):
            self.env.reset(benchmark=self.env.datasets["cbench-v1"].random_benchmark())
            for _ in range(phases):
                action = env.action_space.sample()
                _, reward, done, _ = env.step(action)
                if done: break
                # self.xs.append(torch.tensor(self.env.observation["Inst2vec"]))
                x = np.insert(
                    self.env.observation["Inst2vecEmbeddingIndices"],
                    0, action + offset, axis=0)
                
                t = np.ones_like(x)
                t[0] = 0.0
                
                self.xs.append(torch.tensor(x))
                self.ts.append(torch.tensor(t)) 
                self.ys.append(torch.tensor(reward))
        
    def __len__(self):
        return len(self.xs)

    def __getitem__(self, i):
        # We’ll pad at the batch level.
        return {
            "input_ids": self.xs[i][:512],
            "token_type_ids" : self.ts[i][:512],
            "label": self.ys[i]
        }

In [18]:
# ds = LlvmIrDataset(10, 2)
# ds[0]

In [19]:
# tokenizer = ByteLevelBPETokenizer()
# # tokenizer = AutoTokenizer.from_pretrained("huggingface/CodeBERTa-small-v1")
# tokenizer.train_from_iterator(sampler(), special_tokens=["[SEP]", "[CLS]"])
# tokenizer.save("vocab/tokenizer.json")
# tokenizer.save_model("vocab/")

In [20]:
# # Prepare the tokenizer
# tokenizer._tokenizer.post_processor = BertProcessing(("[SEP]", tokenizer.token_to_id("[SEP]")), ("[CLS]", tokenizer.token_to_id("[CLS]")),)
# tokenizer.enable_truncation(max_length=512)

In [21]:
# inst2vec_vocab = list(env.inst2vec.vocab.keys())
# batchsize = 10
# for i in range(0, len(inst2vec_vocab), batchsize):
#     batch = inst2vec_vocab[i:i+batchsize]
#     tokenizer.add_tokens(batch)

In [22]:
tokenizer = RobertaTokenizerFast.from_pretrained('microsoft/codebert-base-mlm', add_prefix_space=True)
# tokenizer.add_tokens(env.action_space.names)

In [27]:
def sampler(samples=SAMPLES, phases=PHASES):
    for benchmark in env.datasets["cbench-v1"].benchmarks():
        print(benchmark)
        for _ in range(samples):
            env.reset(benchmark=benchmark)
            for phase in range(phases):
                action = env.action_space.sample()
                _, reward, done, info = env.step(action)
                env.action_space.to_string(action)
                if done: break
                yield tokenizer.sep_token.join(env.observation['Inst2vecPreprocessedText'])

In [28]:
for s in sampler(2, 3):
    print(len(s))

localhost:37085 failed to connect to all addresses (4 attempts remaining)
localhost:37085 failed to connect to all addresses (3 attempts remaining)


benchmark://cbench-v1/adpcm


localhost:37085 failed to connect to all addresses (2 attempts remaining)
localhost:37085 failed to connect to all addresses (1 attempt remaining)
localhost:37085 failed to connect to all addresses (0 attempts remaining)
Failed to stop session 60 with ServiceTransportError: localhost:37085 failed to connect to all addresses (5 retries)
localhost:37085 failed to connect to all addresses (4 attempts remaining)
localhost:37085 failed to connect to all addresses (3 attempts remaining)
localhost:37085 failed to connect to all addresses (2 attempts remaining)
localhost:37085 failed to connect to all addresses (1 attempt remaining)
localhost:37085 failed to connect to all addresses (0 attempts remaining)
ServiceTransportError during reset(): localhost:37085 failed to connect to all addresses (5 retries)


ServiceError: Service exited with returncode -2

In [ ]:
tokenizer = tokenizer.train_new_from_iterator(sampler(10, 5),
                                              vocab_size = len(env.inst2vec.vocab),
                                              new_special_tokens = env.action_space.names)

In [ ]:
tokenized = tokenizer(env.observation['Inst2vecPreprocessedText'],
          is_split_into_words=True,
          padding=True,
          truncation=True,
          return_tensors="pt")
tokenized

In [ ]:
tokenized['input_ids'].shape

In [ ]:
tokenizer.sep_token

In [ ]:
len(env.observation['Inst2vecPreprocessedText'])